# Lec5 `#5/12`

## Lazy Evaluation

For **applicative order** evaluation, we had that:

1. Eval subexpressions
2. If primitive proc, just apply it
3. If compound proc, eval body in new env which extends proc.env with new frame that binds proc params to arg vals

In **lazy evaluation (normal order)**:

- We apply proc to unevaluated argument subexpressions.
- Eval a subexpression only when value is needed. 
  - to print
  - primitive procedure (they are 'strict' in their args), e.g, 
  ```scheme
  (foo (+ x y) (- x y))
  ;------------------->   Evaluated in normal order
  ;    ----->             Evaluated in applicative order, not normal order
  ;            ------>    Evaluated in applicative order, not normal order
  ```

#### Exercise 1

In applicative order:

In [1]:
(define (foo x)
  (display "inside foo") (newline) ; dbg
  (+ x x))
(foo (begin 
      (display "eval arg") (newline)  ; dbg
      222))

eval arg
inside foo


444

In lazy order (without caching):
```
inside foo
eval arg
eval arg
444
```

### To Implement lazy eval:

- Need "delayed objects"
- Change eval to force evaluation only when needed


- When is lazy eval useful ?

### Representing delayed objects:

- A thunk is a promise to return a value when needed (forced)
  - --> [ thunk | -]-> [ exp | -]-> [ env | / ]

```scheme
(define (delay-it exp env)
    (list 'thunk exp env))

(define force-it obj)
    (cond ((thunk? obj)
           (actual-value (thunk-exp obj)
                         (thunk-env obj)))
          (else obj)))

(define (actual-value exp env)
    (force-it (eval exp env)))
```

#### Example

```scheme
(define a (delay-it '(+ 2 3) GE))
; a : '(thunk (+ 2 3) GE)
(force-it a) ; 5
```

### Changes in eval/apply for normal order:

- `eval`:
  - does not eval arguments
  - needs to call apply with env
- `apply`:
  - eval args with supplied env if primitive procedure
  - construct a list of delayed args if compound procedure

### When is lazy-eval useful?

- Cons now acts like cons-stream
- `(foo (very-long-computation-1) (very-long-computation2))` may not need to do both calculations
  - If only one is used, lazy is advantageous
- `(foo (very-long-computation-3))` may use the arg more than ocne
  - If the arg is used twice, then applicative is advantageous.
  - Can be run fast in lazy with caching



### Memoizing Thunks

[thunk|-]->[exp|-]->[env|/]

$ \ \ \downarrow $

[evaluated-thunk|-]->[result|/]



